# **Example 1** Search and read EarthCARE data

In [ ]:
import earthcarekit as eck

## Requirements

EarthCARE files:
- `ATL_NOM_1B` frame 01508B
- `ATL_EBD_2A` full orbit 5813 (frames A to H)

Place the file in the notebook's directory or within a subfolder.

If needed, you can (optionally) download the product by executing the following code block:

In [ ]:
print("Download required products:")
print("#######################################")
print("-> Download: ATL_NOM_1B frame 01508B")
eck.ecdownload(
    file_type="ATL_NOM_1B",
    orbit_and_frame="1508B",
    path_to_data=".",  # Select the notebook's folder as root directory
    is_create_subdirs=False,  # Don't create a nested subfolder structure for downloaded files
)
print("#######################################")
print("-> Download: ATL_EBD_2A full orbit 5813")
eck.ecdownload(
    file_type="ATL_EBD_2A",
    orbit_number=5813,
    path_to_data=".",  # Select the notebook's folder as root directory
    is_create_subdirs=False,  # Don't create a nested subfolder structure for downloaded files
)

## **Example 1.1** Search files

In [ ]:
print("Search products using different filters:")

df = eck.search_product(
    file_type="ATL_NOM_1B",
    orbit_and_frame="1508B",
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print("- Results using 'orbit_and_frame':", df.name)

df = eck.search_product(
    file_type="ATL_NOM_1B",
    orbit_number=1508,
    frame_id="B",
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print("- Results using 'orbit_number' and 'frame_id':", df.name)

df = eck.search_product(
    file_type="ATL_NOM_1B",
    timestamp="2024-09-02 21:05", # Returns products containing this timestamp
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print("- Results using 'timestamp':", df.name)

df = eck.search_product(
    file_type="ATL_NOM_1B",
    start_time="20240902T21",
    end_time="20240902T2110",
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print("- Results using 'start_time' and 'end_time':", df.name)

df = eck.search_product(
    file_type=["anom", "aebd"],
    orbit_number=[1508, 5813],
    frame_id=["A", "B", "C"],
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print("- Results using lists:\n", df.name)

## **Example 1.2** Read single files

In [ ]:
print("Search products by orbit number and frame ID:")
df = eck.search_product(
    file_type="ATL_NOM_1B",
    orbit_and_frame="1508B",
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print(df.name)

fp = df.filepath[-1]  # Select file from ProductDataFrame

with eck.read_product(fp) as ds:
    display(ds)
    cf = eck.CurtainFigure()
    cf = cf.ecplot(
        ds=ds,
        var="mie_attenuated_backscatter",
        height_range=(0, 25e3),
    )

## **Example 1.3** Read consecutive files or a full orbit

> 💡 Note: When reading multiple products with `read_products()`, the resulting dataset is coarsened by averaging adjacent pixels. For example, merging 8 products averages every 8 neighboring pixels into one.
>
> Alternatively, you can zoom into a region by setting the `zoom_at` fractional index - without loss of resolution. For example in case you read a full orbit (8 files):
> - `zoom_at=0` will display the 1st frame (A frame)
> - `zoom_at=0.5` will display data from center of the 1st to center of the 2nd frame - with the transition between both frames at the center
> - `zoom_at=1` will display the 2nd frame (B frame)
> - `zoom_at=1.5` transition from 2nd to 3rd frame
> - `zoom_at=2` 3rd frame (C frame)
> - `zoom_at=2.25` last 75% of the 3rd and first 25% of the 4th frame
> - ...

In [ ]:
print("Search products of a full orbit:")
df = eck.search_product(
    file_type="ATL_EBD_2A",
    orbit_number=5813,
    root_dirpath=".",  # Select the notebook's folder as root directory
)
print(df.name)

In [ ]:
print("Read and merge datasets of a full orbit:")
with eck.read_products(df) as ds:
    display(ds)

In [ ]:
print("Plot the full orbit:")
cf = eck.CurtainFigure()
cf = cf.ecplot(
    ds=ds,
    var="particle_backscatter_coefficient_355nm",
    height_range=(0, 25e3),
)

In [ ]:
print("Show only transition between 1st and 2nd frame:")
with eck.read_products(df, zoom_at=0.5) as ds:
    cf = eck.CurtainFigure()
    cf = cf.ecplot(
        ds=ds,
        var="particle_backscatter_coefficient_355nm",
        height_range=(0, 25e3),
    )